## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals

In [3]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [17]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [8]:
Tag_client = AxClient()

[INFO 04-03 15:20:15] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [9]:
# Define bounds for each parameter type
int_bounds = {
    "z_s": [2, 30],
    "z_p2": [25, 35],
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.5, 0.5],
    "x_p1": [-1.0, 1.0],
    "x_p2": [-0.5, 0.5],
    "x_r2": [-1.5, 1.5],
}

# Number of copies for each parameter
num_copies = 3

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": bounds,
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": bounds,
            "value_type": "float",
        })

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Viable_Cross_Terms": ObjectiveProperties(minimize=False)},
    # parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-03 15:20:15] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_s', parameter_type=INT, range=[2, 30]), RangeParameter(name='z_s_1', parameter_type=INT, range=[2, 30]), RangeParameter(name='z_s_2', parameter_type=INT, range=[2, 30]), RangeParameter(name='z_s_3', parameter_type=INT, range=[2, 30]), RangeParameter(name='z_p2', parameter_type=INT, range=[25, 35]), RangeParameter(name='z_p2_1', parameter_type=INT, range=[25, 35]), RangeParameter(name='z_p2_2', parameter_type=INT, range=[25, 35]), RangeParameter(name='z_p2_3', parameter_type=INT, range=[25, 35]), RangeParameter(name='z_r2', parameter_type=INT, range=[70, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[70, 86]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 86]), RangeParameter(name='z_r2_3', parameter_type=INT, range=[70, 86]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.5, 0.5]), RangeParameter(name='x_s_1', parameter_ty

### Run tests to solve for best possible score    

In [10]:
parameterization, trial_index = Tag_client.get_next_trial()

c:\Users\campb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ax\modelbridge\cross_validation.py:439: UserWarning:

Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.

[INFO 04-03 15:20:16] ax.service.ax_client: Generated new trial 0 with parameters {'z_s': 2, 'z_s_1': 15, 'z_s_2': 21, 'z_s_3': 5, 'z_p2': 30, 'z_p2_1': 29, 'z_p2_2': 26, 'z_p2_3': 31, 'z_r2': 83, 'z_r2_1': 75, 'z_r2_2': 74, 'z_r2_3': 72, 'x_s': 0.230362, 'x_s_1': 0.164608, 'x_s_2': -0.336502, 'x_s_3': -0.272492, 'x_p1': -0.240404, 'x_p1_1': -0.57703, 'x_p1_2': -0.681771, 'x_p1_3': -0.449725, 'x_p2': -0.313794, 'x_p2_1': 0.149482, 'x_p2_2': 0.220838, 'x_p2_3': -0.378625, 'x_r2': 0.899606, 'x_r2_1': 0.964305, 'x_r2_2': 0.010575, 'x_r2_3': 0.74046} using model Sobol.


In [12]:
parameterization

{'z_s': 2,
 'z_s_1': 15,
 'z_s_2': 21,
 'z_s_3': 5,
 'z_p2': 30,
 'z_p2_1': 29,
 'z_p2_2': 26,
 'z_p2_3': 31,
 'z_r2': 83,
 'z_r2_1': 75,
 'z_r2_2': 74,
 'z_r2_3': 72,
 'x_s': 0.23036247491836548,
 'x_s_1': 0.16460782289505005,
 'x_s_2': -0.33650214970111847,
 'x_s_3': -0.27249205112457275,
 'x_p1': -0.24040448665618896,
 'x_p1_1': -0.577029824256897,
 'x_p1_2': -0.6817713379859924,
 'x_p1_3': -0.4497247338294983,
 'x_p2': -0.31379425525665283,
 'x_p2_1': 0.14948159456253052,
 'x_p2_2': 0.2208382487297058,
 'x_p2_3': -0.3786247745156288,
 'x_r2': 0.899606466293335,
 'x_r2_1': 0.9643049240112305,
 'x_r2_2': 0.010574698448181152,
 'x_r2_3': 0.7404600977897644}

In [ ]:
for i in range(13):
    parameterization, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameterization))